# Big Data - Dashboard
## ALBICHARI Kaïs - D'HOSE Tanguy - ULB

In [6]:
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
import sys
import os
from datetime import datetime
from pathlib import Path
from kafka import KafkaConsumer
from kafka.errors import KafkaError

In [1]:
import headtail
headtail.head('data/data.conv.txt', 5)

['2017-03-31 03:38:16.508 1-0 122.153 2.03397\n',
 '2017-03-31 03:38:15.967 1-1 -3.91901 2.09397\n',
 '2017-03-31 03:38:16.577 1-2 11.04 2.07397\n',
 '2017-02-28 00:59:16.359 1-0 19.9884 2.74964\n',
 '2017-02-28 00:59:16.803 1-1 37.0933 2.76964\n']

In [5]:
sc = SparkContext("local[*]", "test")
sc.setLogLevel("WARN")   #Make sure warnings and errors observed by spark are printed.

ssc = StreamingContext(sc, 5)  #generate a mini-batch every 5 seconds

In [ ]:
def checkAttributes(sensor_type, space_tag, time_tag):
    if sensor_type < 0 or sensor_type > 3:
        print("Sensor type value is not supported. Give a value in [0,3]")
        return False
    elif space_tag < 0 or space_tag > 2:
        print("Space tag value is not supported. Give a value in [0,2].")
        print("0: grouped per space")
        print("1: grouped per municipality")
        print("2: grouped for the entirety of Brussels")
        return False
    elif time_tag <0 or time_tag > 4:
        print("Time tag value is not supported. Give a value in [0,4].")
        print("0: last 24h")
        print("1: last 2 days")
        print("2: last week")
        print("3: last month")
        print("4: last year")
        return False
    else:
        return True
        

def basicStats(sensor_type, space_tag, time_tag):
    """
    Basic statistics (min, max, avg) about sensor readings grouped according to different granularities
    in space and time.
    sensor_type: type of the sensor (0 to 3)
    space_tag: defines granularity in space (0 to 2)
    time_tag: defines granularity in time (0 to 4)
    """
    if checkAttributes(sensor_type, space_tag, time_tag):
        try:
            consumer = KafkaConsumer(bootstrap_servers = ['localhost:9092'])
            topic = "1010-2"# = "*-"+str(sensor_type)
            consumer.subscribe([topic])
            for i in range (5):
                row = next(consumer)
                print(row.value.decode())

        except Exception as err:
             print("Unexpected error: %s" % (err))
        finally:
            consumer.close()


basicStats(1,2,3)
    

In [29]:
try:
    consumer = KafkaConsumer(bootstrap_servers = ['localhost:9092'])
    topic = "1000-1"
    pattern = ".-1"
    consumer.subscribe(pattern = pattern)
    for i in range (5):
        row = next(consumer)
        print(row)

except Exception as err:
     print("Unexpected error: %s" % (err))
finally:
    consumer.close()

KeyboardInterrupt: 

In [23]:
def parseOrder(line):
    '''parses a single line in the orders file into a dictionary'''
    s = line.split(",")
    try:
        if s[6] != "B" and s[6] != "S":
            raise Exception('Wrong format')
        return [{"time": datetime.strptime(s[0], "%Y-%m-%d %H:%M:%S"),
                   "orderId": int(s[1]), 
                   "clientId": int(s[2]),
                   "symbol": s[3], 
                   "amount": int(s[4]), 
                   "price":  float(s[5]), 
                   "buy": "BUY" if s[6] == "B" else "SELL"}]
    except Exception as err:
        print("Wrong line format (%s): %s" % (line,err))